In [4]:
import pandas as pd
import datetime

In [6]:
data_ = pd.read_csv('..\data\charts.csv')
data_.head()

In [50]:
data = data_.iloc[:29999]

In [7]:
# Artist
df_artist = pd.DataFrame(data['artist'].unique(), columns=['name'])
df_artist = df_artist.assign(name=df_artist['name'].str.split(', ')).explode('name')
df_artist = pd.DataFrame(df_artist['name'].unique(), columns=['name'])
df_artist.reset_index(drop = True, inplace=True)
df_artist.insert(0, 'artist_id', range(1, 1 + len(df_artist)))

# Song
df_song = pd.DataFrame(data['title'].unique(), columns=['title'])
df_song.insert(0, 'song_id', range(1, 1 + len(df_song)))

# Trend
df_trend = pd.DataFrame(data['trend'].unique(), columns=['trend'])
df_trend.insert(0, 'trend_id', range(1, 1 + len(df_trend)))

# Day
df_day = pd.DataFrame(data['date'].unique(), columns=['date'])
df_day.insert(0, 'day_id', range(1, 1 + len(df_day)))

# Region
df_region = pd.DataFrame(data['region'].unique(), columns=['name'])
df_region.insert(0, 'region_id', range(1, 1 + len(df_region)))

# Category
df_category = pd.DataFrame(data['chart'].unique(), columns=['name'])
df_category.insert(0, 'category_id', range(1, 1 + len(df_category)))


In [8]:
artistDict = df_artist.set_index('name').T.to_dict('records')[0]
songDict = df_song.set_index('title').T.to_dict('records')[0]
trendDict = df_trend.set_index('trend').T.to_dict('records')[0]
dayDict = df_day.set_index('date').T.to_dict('records')[0]
regionDict = df_region.set_index('name').T.to_dict('records')[0]
categoryDict = df_category.set_index('name').T.to_dict('records')[0]

In [51]:
batch = 10000
batches = int(len(data)/batch)
residual = len(data) - batches*batch

print(f"Data len: {len(data)} | Batch: {batch} | Batches: {batches} | Residual {residual}")

Data len: 29999 | Batch: 10000 | Batches: 2 | Residual 9999


In [52]:
df_chart_final = pd.DataFrame()
num = 0
# MAIN BATCHES
start = datetime.datetime.now().timestamp()
for j in range(batches):
    df_chart = pd.DataFrame(columns=[ 'title','rank', 'date', 'region', 'chart', 'trend', 'streams'])
    start = datetime.datetime.now().timestamp()
    for i in range(batch):
        df_row = data.iloc[num]
        title = songDict[df_row.iloc[0]]
        rank = df_row.iloc[1]
        date = dayDict[df_row.iloc[2]]
        region = regionDict[df_row.iloc[5]]
        chart = categoryDict[df_row.iloc[6]]
        trend = trendDict[df_row.iloc[7]]
        streams = df_row.iloc[8]
        row = [title, rank, date, region, chart, trend, streams]
        df_chart.loc[num] = row
    
        num +=1
        pass
    df_chart_final = pd.concat([df_chart_final, df_chart])
    
    print(f"Batch {j+1}/{batches} done [{datetime.datetime.now().timestamp()-start}]")

# RESIDUALS
start = datetime.datetime.now().timestamp()
df_chart = pd.DataFrame(columns=[ 'title','rank', 'date', 'region', 'chart', 'trend', 'streams'])
for i in range(residual):
    df_row = data.iloc[num]
    title = songDict[df_row.iloc[0]]
    rank = df_row.iloc[1]
    date = dayDict[df_row.iloc[2]]
    region = regionDict[df_row.iloc[5]]
    chart = categoryDict[df_row.iloc[6]]
    trend = trendDict[df_row.iloc[7]]
    streams = df_row.iloc[8]
    row = [title, rank, date, region, chart, trend, streams]
    df_chart.loc[num] = row

    num +=1
    pass
df_chart_final = pd.concat([df_chart_final, df_chart])

print(f"Batch residual done [{datetime.datetime.now().timestamp()-start}]")
df_chart_final.to_csv('../data/df_chart.csv', mode='w', index = 'False')
print(df_chart_final)

Batch 1/2 done [13.49668002128601]
Batch 2/2 done [12.640635967254639]
Batch residual done [12.743932008743286]
        title   rank  date  region  chart  trend   streams
0         1.0    1.0   1.0     1.0    1.0    1.0  253019.0
1         2.0    2.0   1.0     1.0    1.0    2.0  223988.0
2         3.0    3.0   1.0     1.0    1.0    3.0  210943.0
3         4.0    4.0   1.0     1.0    1.0    1.0  173865.0
4         5.0    5.0   1.0     1.0    1.0    2.0  153956.0
...       ...    ...   ...     ...    ...    ...       ...
29994   245.0  192.0   6.0    11.0    1.0    3.0    8561.0
29995   721.0  193.0   6.0    11.0    1.0    3.0    8558.0
29996  5248.0  194.0   6.0    11.0    1.0    3.0    8531.0
29997   454.0  195.0   6.0    11.0    1.0    3.0    8471.0
29998   184.0  196.0   6.0    11.0    1.0    3.0    8425.0

[29999 rows x 7 columns]
